In [1]:
import numpy as np
import random as rn
import findspark
import sys
import time
import numba
from numba import cuda, float32, float64
findspark.init("/opt/spark")

In [2]:
from pyspark import SparkContext
sc = SparkContext("local","PSO Distribuido")

In [3]:
def MSE(y, pred):
    n = len(y)
    if n != len(pred):
        print("error: datos y predicción de distintos tamaños")
        return -1

    resultado = 0.0

    for i in range(n):
        resultado += (y[i] - pred[i])**2

    resultado = resultado / n

    return resultado

In [34]:
V_max= 10.0
W= 1.0
c_1 = 0.8
c_2 = 0.2
objetivo =[50,50,50]
ind=0
filas_x=[0,0,0]

In [35]:
#Número dimensiones de los vectores
n=3
#Número de partículas
m= 10
#Número de iteraciones
I = 100
posiciones_=[]
mejor_posiciones_locales_=[]
velocidades_=[]
particulas = np.array([posiciones_,velocidades_,mejor_posiciones_locales_])
mejor_pos_global_=[]
mejor_pos_global_arr= np.array([0 for j in range(n)])
best_local_fitness_= []
best_local_fitness_arr=np.array(best_local_fitness_)
#maximum float
max = sys.float_info.max
best_global_fitness = max
accum=sc.accumulator(0)
#accum=sc.accumulator(0.0)
#mejor_pos_global= sc.broadcast(mejor_pos_global_arr)
mejor_pos_global= mejor_pos_global_arr
best_local_fitness= sc.broadcast(best_local_fitness_arr)

In [36]:
def InitParticles(part,N,M,bgf,blf):
    #global best_global_fitness
    parts_ =[]
    for j in range(M):
        posicion=[rn.uniform(-100,100) for j in range(N)]
        velocidad=[rn.uniform(-100,100) for j in range(N)]
        fit = MSE(posicion,objetivo)
        #posición, velocidad, mejor posición local, mejor fitness local
        part_ = posicion + velocidad + posicion + [fit]
        parts_.append(part_)

        best_local_fitness_.append(fit)
        blf= np.array(best_local_fitness_)
        #print("bgf-> ",bgf)
        if fit < bgf:
            bgf = fit
            accum= fit
            mejor_pos_global = posicion
    parts = np.array(parts_)
    return blf,bgf,mejor_pos_global,parts 

In [37]:
def fitnessEval(part,N):
    global mejor_pos_global
    global best_global_fitness
    global accum
    global best_local_fitness
    best_fit_local = part[3*N:]
    filas=part[0:N]
    fit = MSE(filas,objetivo)
    if fit < best_fit_local:
        part[3*N:] = fit
        #mpl[ind] = part[0:1,ind]
        for k in range(N):
            part[2*N+k]=filas[k]
        #if fit < best_global_fitness:
            #accum += (fit-best_global_fitness)
            #best_global_fitness = fit
            #mejor_pos_global := filas
            #accum := fit
    #if ind<10:
        #ind+=1
    #return part,mpg,blf,bgf
    return part 

In [38]:
def posEval(part,mpg,N):
#def posEval_x(part,N):
    global ind
    velocidades=part[N:2*N]
    mpl=part[2*N:]
    r_1 = rn.random()
    r_2 = rn.random()
    for k in range(N):
        velocidades[k] = W*velocidades[k] + c_1*r_1*(mpl[k] - part[k]) + c_2*r_2*(mpg[k] - part[k])
        if velocidades[k] > V_max:
            velocidades[k] = V_max
        elif velocidades[k] < -V_max:
            velocidades[k] = -V_max  
        part[k] = part[k] + velocidades[k]
        part[N+k] = velocidades[k]
    #return part,velocidades
    return part

In [39]:
best_local_fitness,best_global_fitness,mejor_pos_global,particulas=InitParticles(particulas,n,m,best_global_fitness,best_local_fitness)

In [47]:
rdd_master= sc.parallelize(particulas)
start = time.time()

print("rdd_master antes:", rdd_master.collect())
print("antes:",mejor_pos_global)
for i in range(100):
    rdd_2 = rdd_master.map(lambda x:fitnessEval(x,n))
    rdd_eval = rdd_2.collect()
    
    for j in range(m):
        blf = rdd_eval[j][3*n:]
        if blf < best_global_fitness:
            best_global_fitness = blf
            mejor_pos_global = rdd_eval[j][0:n]
    
    
    
    resultado = rdd_2.map(lambda x:posEval(x,mejor_pos_global,n))
    resultado = resultado.collect()
    
    #for j in range(m):
    #    particulas[j] = resultado[j][:]
        
    rdd_master= sc.parallelize(resultado)
    
print("despues:",mejor_pos_global)
print("rdd_master despues:", rdd_master.collect())

end = time.time()
tiempo = end - start
resultado = rdd_master.collect()
print("Tiempo de ejecucion(s): ", tiempo)
print("mejor_pos_global-> ",mejor_pos_global)
print("mejor fitness global-> ", best_global_fitness, MSE(mejor_pos_global,objetivo))
print("accum:", accum.value)

rdd_master antes: [array([ -21.10804658,   64.54204823,  -98.87070322,   39.85906318,
        -64.93048598,  -80.32198252,  -21.10804658,   64.54204823,
        -98.87070322, 9143.43724441]), array([  -68.3570251 ,   -89.56835405,    98.95265116,    18.13677294,
          16.5804239 ,   -81.60616912,   -68.3570251 ,   -89.56835405,
          98.95265116, 11961.35763273]), array([ -58.56619867,   -6.3469998 ,    6.60322075,  -12.65230311,
        -59.45580569,   92.9148525 ,  -58.56619867,   -6.3469998 ,
          6.60322075, 5614.96144347]), array([ -18.40726181,   70.68458028,  -95.67903137,   70.12630093,
         66.38319698,   21.47833772,  -18.40726181,   70.68458028,
        -95.67903137, 8776.59517   ]), array([ -83.57568126,   35.51160324,   24.45297744,   22.45495142,
          9.56566996,  -63.24148204,  -83.57568126,   35.51160324,
         24.45297744, 6235.00887584]), array([  82.54115885,   43.78446427,  -38.62625611,   69.4362102 ,
        -35.09821268,  -71.0429739 ,   